# Games Recommender System

This is a recommender system based on Stream user information. We need to extract all data needed from steamAPI and then build models to find the appropriate games to recommend. 

In [80]:
import requests, json, os, sys, time, re
from datetime import datetime
from multiprocessing import Pool # multiprocess function for each worker using
import requests
from bs4 import BeautifulSoup
import math
import pandas as pd
import pymysql
import sqlalchemy
import string
from sklearn.feature_extraction.text import TfidfVectorizer # Process text infomation
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity # Calculate the similar users or products
import numpy as np


In [81]:
# Loading bar
def show_work_status(singleCount, totalCount, currentCount=0):
    currentCount += singleCount
    percentage = 1. * currentCount / totalCount * 100
    status =  '>' * int(percentage)  + ' ' * (100 - int(percentage))
    sys.stdout.write('\rStatus: [{0}] {1:.2f}% '.format(status, percentage))
    sys.stdout.flush()
    if percentage >= 100:
        print ('\n')

# Split a long list into n short lists
def split_list(lst_long,n):
    lst_splitted = []
    if len(lst_long) % n == 0:
        totalBatches = math.floor(len(lst_long) / n)
    else:
        totalBatches = math.floor(len(lst_user_id) / n + 1)
    for i in range(totalBatches):
        lst_short = lst_long[i*n:(i+1)*n]
        lst_splitted.append(lst_short)
    return lst_splitted

## Data ETL

#### 1. Crawl users' inventory
Crawling user infomation by user id and store it into txt file(user_inventory)

GetOwnedGames (v0001):

GetOwnedGames returns a list of games a player owns along with some playtime information, if the profile is publicly visible. Private, friends-only, and other privacy settings are not supported unless you are asking for your own personal details (ie the WebAPI key you are using is linked to the steamid you are requesting).

https://developer.valvesoftware.com/wiki/Steam_Web_API#GetOwnedGames_.28v0001.29

In [3]:
# Read the user id in txt file
path_user_id = 'data/steam_user_id.txt'
# Open txt file and read binary since the id is effective for every 2 lines
with open(path_user_id, 'rb') as f:
#     Eliminate n/t/ at the end of every id
#     Read first 50
    lst_user_id = [x.strip() for x in f.readlines()]

In [4]:
len(lst_user_id)

5000

In [140]:
def worker(lst_user_id_temp):
#     Empty dict for store id:contents
    dic_temp = {}
    for user_id in lst_user_id_temp[0:len(lst_user_id_temp)]:
#         API website
        base_url = "http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/"
#         Keys API requred
        params = {
            'key' : 'D4BA51BB9B8BEF48E66C731BB70B7BCF',
            'steamid' : user_id.strip(),
            'format' : 'json' }
        r = requests.get(base_url, params = params)
#         Get games information 
        user_inventory = r.json(encoding = 'utf-8').get('response').get('games')
#         Update dictionary for contents scraped using id
        dic_temp.update({user_id:user_inventory})
#         Sleep 0.5 second to avoid causing stress for the website
        time.sleep(.5)
    return dic_temp

In [12]:
# 2 Pools for 50 id means 2 workers process 50 ids seperately, each handles 25 ids.
p = Pool(2)

# Parameter for loading par
total_count = len(lst_user_id)
current_count = 0
show_work_status(0, total_count, current_count)

# Creat master dict to combine the total works handled by 2 pools
dic_master = {}
# Process a batch(500 ids) every time
for i in split_list(lst_user_id,10):
#   2 workers works on 1/2 batch(100 ids) each time
    lst_temp_dic = p.map(worker, split_list(i,5))
#   Update dic_master after getting the id and its content
    for j in lst_temp_dic:
        dic_master.update(j)
#   Show the work status for every batch processed
    show_work_status(len(i), total_count, current_count)
    current_count += len(i)
#   In case of causing stress to website
    time.sleep(0.5)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



Process ForkPoolWorker-6:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/anaconda/lib/python3.6/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/anaconda/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/anaconda/lib/python3.

In [14]:
# Store user info into crawled_user_inventory
with open('data/user_inventory.txt', 'w') as f:
    for user_id, user_inventory in list(dic_master.items()):
        f.write(json.dumps({str(user_id):user_inventory}))
        f.write('\n')

#### 2.1 Get app ID

Get all app's information(id, price, userscore...) from steam API, 64342 app ids in total.

In [4]:
# Get all app's ID from steam

url_app = 'http://api.steampowered.com/ISteamApps/GetAppList/v2/'
r = requests.get(url_app)
dic_app_list = r.json()
lst_app_id = [i.get('appid') for i in dic_app_list.get('applist').get('apps')]

In [5]:
len(lst_app_id)

66424

In [6]:
lst_app_id[0:5]

[5, 7, 8, 10, 20]

#### 2.2 Scrape the all app information using app ID

Store the information into path_app_detail.

In [8]:
total_count = len(lst_app_id)#lst_app_id)
current_count = 0
show_work_status(0, total_count, current_count)

# Create text file for all app detailed information.
path_app_detail = 'data/path_app_detail.txt'

# Prepare file to be written
with open(path_app_detail, 'w') as f:
    for app_id in lst_app_id:
 
        url_app_detail = ('http://store.steampowered.com/api/appdetails?appids=%s') % (app_id) # Scrape using appID
        
        # Get the app detail and transfer to json format, if the app ID is not readable through api website, pass it
        for i in range(2):
            try:
                r = requests.get(url_app_detail)
                result = r.json()
                break
            except:
                time.sleep(5)
                pass
        f.write(json.dumps(result))
        f.write('\n')
        show_work_status(1, total_count, current_count)
        current_count += 1
        if current_count % 200 == 0:
            time.sleep(2.5)
        else:
            time.sleep(.5)

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



Extract the detailed game information and the description of game

In [9]:
# Open and read the scraped app information
with open(path_app_detail, 'r') as f:
#     Create dic store information for each app id
    dic_steam_app = {'initial_price':{},'name':{},'score':{},'windows':{},'mac':{},'linux':{},'type':{},'release_date':{},'recommendation':{},'header_image':{}}
    dic_about_the_game = {}
#     Read txt file into lst_raw_string
    lst_raw_string = f.readlines()
#     Set counts for statue bar
    total_count = len(lst_raw_string)
    current_count = 0
    show_work_status(0, total_count, current_count)
    
#     release_date = ''
#     Read each item in the list of app information
    for raw_string in lst_raw_string:
        try:
#           Only one item in the list, extract the item from the list
            app_data = list(json.loads(raw_string).values())[0]
        except:
            pass
        
        if app_data != {} and app_data.get('success') != False: # If app_data is not empty

#           Get needed information from app_data
            steam_id = app_data.get('data').get('steam_appid') # Get game id
            initial_price = app_data.get('data').get('price_overview',{}).get('initial')
            if app_data.get('data').get('is_free') == True: # Get price
                initial_price = 0
                
            app_name = app_data.get('data').get('name') # Get game name
            
            critic_score = app_data.get('data').get('metacritic',{}).get('score') # Get score
            
            app_type = app_data.get('data').get('type') # Get game type
            
            for (platform, is_supported) in app_data.get('data').get('platforms',{}).items():
                if is_supported == True: # Get platform game runs on
                    dic_steam_app[platform].update({steam_id:1})
            
            if app_data.get('data').get('release_date',{}).get('coming_soon') == False: # Get date
                release_date = app_data.get('data').get('release_date',{}).get('date')
                if not release_date == '':
                    try:
                        release_date = datetime.strptime(release_date, '%b %d, %Y')
                    except:
                        try:
                            release_date = datetime.strptime(release_date, '%d %b, %Y')
                        except:
                            release_date = None
            
            about_the_game = app_data.get('data').get('about_the_game')# Get game description
            soup = BeautifulSoup(about_the_game,'lxml') # Tranfer text to lxml to remove lxml format in text
            game_description = re.sub(r'(\s+)',' ',soup.text).strip() # re.sub(pattern, repl, string) Reduce \t,\n
            dic_about_the_game.update({steam_id:game_description})
            
            recommendation = app_data.get('data').get('recommendations',{}).get('total') # Get recommendation
            header_image = app_data.get('data').get('header_image') # Get path of header image
            dic_steam_app['recommendation'].update({steam_id:recommendation})
            dic_steam_app['header_image'].update({steam_id:header_image})
            
            dic_steam_app['initial_price'].update({steam_id:initial_price})
            dic_steam_app['name'].update({steam_id:app_name})
            dic_steam_app['score'].update({steam_id:critic_score})
            dic_steam_app['type'].update({steam_id:app_type})
            dic_steam_app['release_date'].update({steam_id:release_date})

            

                
        show_work_status(1, total_count, current_count)
        current_count += 1

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [10]:
dic_about_the_game[10]

"Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role."

#### 3. Transfer scraped app information into dataframe

In [31]:
df_steam_app = pd.DataFrame(dic_steam_app)
# Transfer price to normal price
df_steam_app.initial_price = df_steam_app.initial_price.map(lambda x: x/100.0)
df_steam_app.index.name = 'steam_appid'
df_steam_app['windows'] = df_steam_app.windows.fillna(0)
df_steam_app['mac'] = df_steam_app.mac.fillna(0)
df_steam_app['linux'] = df_steam_app.linux.fillna(0)
df_steam_app = df_steam_app[['name', 'type', 'initial_price', 'release_date', 'score', 'recommendation', 'windows', 'mac', 'linux', 'header_image']]
df_steam_app.reset_index(inplace=True)
df_steam_app.to_csv('data/path_steam_app_info',encoding='utf8',index=False)

In [11]:
df_steam_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53933 entries, 0 to 53932
Data columns (total 11 columns):
steam_appid       53933 non-null int64
name              53933 non-null object
type              53933 non-null object
initial_price     47033 non-null float64
release_date      53413 non-null datetime64[ns]
score             3313 non-null float64
recommendation    8403 non-null float64
windows           53933 non-null float64
mac               53933 non-null float64
linux             53933 non-null float64
header_image      53933 non-null object
dtypes: datetime64[ns](1), float64(6), int64(1), object(3)
memory usage: 4.5+ MB


In [32]:
df_steam_app.head()

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
0,10,Counter-Strike,game,9.99,2000-11-01,88.0,117473.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/10/...
1,20,Team Fortress Classic,game,4.99,1999-04-01,NaN,3653.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/20/...
2,30,Day of Defeat,game,4.99,2003-05-01,79.0,3576.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/30/...
3,40,Deathmatch Classic,game,4.99,2001-06-01,NaN,1439.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/40/...
4,50,Half-Life: Opposing Force,game,4.99,1999-11-01,NaN,4987.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/50/...


### Connect to SQL 

####  Connect with sqlalchemy and upload dataframe

In [21]:
# mysql+pymysql://<username>:<password>@<host>/<dbname>[?<options>]
engine = sqlalchemy.create_engine('mysql+pymysql://<kellyshaomai@gmail.com>:<Mysql1992>@localhost:3306/gamerec?unix_socket=/tmp/mysql.sock&charset=utf8mb4')
                                

In [52]:
df_steam_app.to_sql('tbl_app_info',engine,if_exists='replace')

In [53]:
results = engine.execute("select * from tbl_app_info limit 5")
for row in results:
    print(row)

(0, 10, 'Counter-Strike', 'game', 9.99, datetime.datetime(2000, 11, 1, 0, 0), 88.0, 116029.0, 1.0, 1.0, 1.0, 'https://steamcdn-a.akamaihd.net/steam/apps/10/header.jpg?t=1528733245')
(1, 20, 'Team Fortress Classic', 'game', 4.99, datetime.datetime(1999, 4, 1, 0, 0), None, 3625.0, 1.0, 1.0, 1.0, 'https://steamcdn-a.akamaihd.net/steam/apps/20/header.jpg?t=1528732825')
(2, 30, 'Day of Defeat', 'game', 4.99, datetime.datetime(2003, 5, 1, 0, 0), 79.0, 3530.0, 1.0, 1.0, 1.0, 'https://steamcdn-a.akamaihd.net/steam/apps/30/header.jpg?t=1512413490')
(3, 40, 'Deathmatch Classic', 'game', 4.99, datetime.datetime(2001, 6, 1, 0, 0), None, 1412.0, 1.0, 1.0, 1.0, 'https://steamcdn-a.akamaihd.net/steam/apps/40/header.jpg?t=1528733362')
(4, 50, 'Half-Life: Opposing Force', 'game', 4.99, datetime.datetime(1999, 11, 1, 0, 0), None, 4876.0, 1.0, 1.0, 1.0, 'https://steamcdn-a.akamaihd.net/steam/apps/50/header.jpg?t=1530046417')


#### 4. Prepare the usage data
Find Most Played Games Per User from steamSPY API.


In [14]:
# Set path
path_app_detail = 'data/path_app_detail.txt' # Scraped info using appID
path_app_stats = 'data/app_stat.json' # Scraped info using appID
path_user_inventory = 'data/user_inventory.txt' # Scraped info using userID

In [15]:
with open(path_app_stats,'rb') as f:
#     read from json file
    dic_steamspy = json.load(f)

In [16]:
dic_steamspy

{'570': {'appid': 570,
  'average_2weeks': 1161,
  'average_forever': 11670,
  'developer': 'Valve',
  'median_2weeks': 614,
  'median_forever': 258,
  'name': 'Dota 2',
  'negative': 104879,
  'owners': 120750179,
  'owners_variance': 302837,
  'players_2weeks': 9008067,
  'players_2weeks_variance': 94228,
  'players_forever': 120750179,
  'players_forever_variance': 302837,
  'positive': 756170,
  'price': '0',
  'publisher': 'Valve',
  'score_rank': 64,
  'userscore': 87},
 '440': {'appid': 440,
  'average_2weeks': 689,
  'average_forever': 4509,
  'developer': 'Valve',
  'median_2weeks': 403,
  'median_forever': 218,
  'name': 'Team Fortress 2',
  'negative': 30274,
  'owners': 44290079,
  'owners_variance': 201228,
  'players_2weeks': 1689127,
  'players_2weeks_variance': 41109,
  'players_forever': 44290079,
  'players_forever_variance': 201228,
  'positive': 464877,
  'price': '0',
  'publisher': 'Valve',
  'score_rank': 86,
  'userscore': 93},
 '730': {'appid': 730,
  'average_

Extract users' favorite game id and get id with the most played game.

In [17]:
dic_user_favorite_app = {}
dic_user_favorite_app_time = {}
with open(path_user_inventory, 'r') as f:
    for raw_string in f.readlines(): # Read each line in the file
        user_id, lst_inventory = list(json.loads(raw_string).items())[0] # Seperate userid&inventory
        if lst_inventory != None and lst_inventory != []:
#             Sort the most played game and extract game id in inventory
            most_played_app_id = sorted(lst_inventory, key=lambda k: k['playtime_forever'])[-1].get('appid')
            most_played_app_id_played_time = sorted(lst_inventory, key = lambda k:k['playtime_forever'])[-1].get('playtime_forever')
        else:
            most_played_app_id = None
        dic_user_favorite_app.update({user_id:most_played_app_id})
        dic_user_favorite_app_time.update({user_id:most_played_app_id_played_time})
        

# Transfter infomation to DF
df_user_favorite_app = pd.Series(dic_user_favorite_app).to_frame().reset_index()
df_user_favorite_app.columns = ['steam_user_id','most_played_app']

df_user_favorite_app_time = pd.Series(dic_user_favorite_app_time).to_frame().reset_index()
df_user_favorite_app_time.columns = ['steam_user_id','most_played_app_time']


In [18]:
df_user_favorite_app_time.head()

,steam_user_id,most_played_app_time
0,76561197960302536,18670
1,76561197960323774,81657
2,76561197960355015,11736
3,76561197960385706,13088
4,76561197960422789,59357


In [19]:
df_user_favorite_app.head()

,steam_user_id,most_played_app
0,76561197960302536,NaN
1,76561197960323774,NaN
2,76561197960355015,10.0
3,76561197960385706,252950.0
4,76561197960422789,730.0


In [25]:
df_user_favorite_app.to_sql('tbl_user_favorite_app', engine, if_exists='replace', index=False)

Only choose the game and the initial price, release date is before today and has initial price.

In [68]:
# Read table from sql
df_steam_app = pd.read_sql('tbl_app_info',engine)
# choose those type is game, release_date <= today and has the price information
df_valid_games = df_steam_app.query('type == "game" and release_date <= "{}" and initial_price >= 0'.format(datetime.today().date().isoformat()))
# Extract the unique steam_appid from df
set_valid_game_id = set(df_valid_games.steam_appid)

In [69]:
df_valid_games.head()

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
0,10,Counter-Strike,game,9.99,2000-11-01,88.0,117473.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/10/...
1,20,Team Fortress Classic,game,4.99,1999-04-01,NaN,3653.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/20/...
2,30,Day of Defeat,game,4.99,2003-05-01,79.0,3576.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/30/...
3,40,Deathmatch Classic,game,4.99,2001-06-01,NaN,1439.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/40/...
4,50,Half-Life: Opposing Force,game,4.99,1999-11-01,NaN,4987.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/50/...


## Build Model

### Model 1 Popularity based

In [22]:
# list(dic_steamspy): all app IDs
# get(app id).get('owners'): how many owners for each game
dic_steamspy.get('570').get('owners')

120750179

In [26]:
df_popularity_based_results = pd.Series([dic_steamspy.get(x).get('owners') for x in list(dic_steamspy)],list(dic_steamspy)).to_frame()
df_popularity_based_results.reset_index(inplace=True)
df_popularity_based_results.columns = ['steam_appid', 'owners']

In [73]:
df_popularity_based_results.to_sql('tbl_results_popularity_based',engine,if_exists='replace')

The most popular games

In [65]:
df_popularity_based_results.head()

,steam_appid,owners
0,570,120750179
1,440,44290079
2,730,39311626
3,304930,35011290
4,578080,27699958


In [63]:
top10games = [int(x) for x in list(df_popularity_based_results.head(11).steam_appid)]

In [64]:
df_steam_app[df_steam_app['steam_appid'].isin(top10games)]

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
17,340,Half-Life 2: Lost Coast,game,0.00,2005-10-27,NaN,6338.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/340...
23,440,Team Fortress 2,game,0.00,2007-10-10,92.0,521238.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/440...
25,550,Left 4 Dead 2,game,9.99,2009-11-16,89.0,237612.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/550...
26,570,Dota 2,game,0.00,2013-07-09,90.0,920096.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/570...
30,730,Counter-Strike: Global Offensive,game,0.00,2012-08-21,83.0,2677551.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/730...
4488,218620,PAYDAY 2,game,9.99,2013-08-13,79.0,348546.0,1.0,0.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/218...
4936,227940,Heroes & Generals,game,0.00,2016-10-18,NaN,92545.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/227...
4987,230410,Warframe,game,0.00,2013-03-25,71.0,198742.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/230...
15711,444090,Paladins®,game,0.00,2018-05-08,83.0,182062.0,1.0,1.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/444...
24748,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,game,29.99,2017-12-21,86.0,865502.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/578...


### Model 2 Content based - Description

Find the most similar game based on description.

#### TfidfVectorizer - Term Frequency – Inverse Document Frequency

TfidfVectorizer is the combination of countVectorizer and TfidfTransformer.

countVectorizer: 

function generates a dictionary which counts the time frequency of each words.
At the same time, use CountVectorizer().fit_transform(wordDictionary) to transfer frequency to metrix array.


TfidfTransformer: 
Some words shows many times but are not meaningful. Some words shows only one time but are very important. We need TfidfTransformer to add weight on features so get a balance.

In [66]:
# all descriptions，without IDs
dic_about_the_game.values()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [70]:
# Intersect id in valid game and game id which has about_the_game description 
# Now, about_the_game only contains the valid game's id
for i in list(set(dic_about_the_game.keys()) - set(df_valid_games.steam_appid)):
    del dic_about_the_game[i]

#  TFIDF to transform values to matrix (frequency of weighted words)
tfidf = TfidfVectorizer(strip_accents='unicode',stop_words='english').fit_transform(list(dic_about_the_game.values()))


/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [72]:
tfidf

<20793x113469 sparse matrix of type '<class 'numpy.float64'>'
	with 1926757 stored elements in Compressed Sparse Row format>

In [71]:
# Have 20793 words
tfidf.shape

(20793, 113469)

#### Cosine similarity
For cosine close to 1 (0 <= cosine <=1), then theta is more likely to be 0. The distance of A and B is shorter so that A is more similar to B. 

Here we use linear similarity which calculate faster than cosine similarity. Liner similarity has no range. When it is smaller, the two matrix are more similar. 




In [540]:
cosine_similarities

array([0., 1., 0., ..., 0., 0., 9.])

In [75]:

print('Content Based Model')

# Find the similarity by the overlapping of words in each game's description
dic_recomended = {}
total_count = len(lst_app_id)
current_count = 0

lst_app_id = list(dic_about_the_game.keys())

# 22887 rows in tfidf
for index in range(tfidf.shape[0]):
#     Calculate the each description's similarity to the whole set
#     flatten() make sure all item in the same list
    cosine_similarities = linear_kernel(tfidf[index:index+1], tfidf).flatten()
#     There are 23 items in the list, sort the index of 23 items
    related_docs_indices = cosine_similarities.argsort()[-2:-22:-1]
#     Append the most similar ids to each id in dic_about_the_game, 
    dic_recomended.update({lst_app_id[index]:[lst_app_id[i] for i in related_docs_indices]})
    show_work_status(1,total_count,current_count)
    current_count+=1

Content Based Model
Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [76]:
# Transfer result to dataframe
df_content_based_results = pd.DataFrame(dic_recomended).T
df_content_based_results.index.name = 'steam_appid'
df_content_based_results.reset_index(inplace=True)
df_content_based_results.to_sql('tbl_results_content_based',engine,if_exists='replace')

From the df we can see the most similar games for each app

In [82]:
df_content_based_results.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,10,20,887520,837940,207230,683600,222880,670270,463920,743230,...,344340,19830,246820,652660,504460,763430,777850,803140,7940,843520
1,20,440,393410,207230,2720,209270,10,485310,511800,99300,...,342660,65800,222940,707010,288710,426190,473810,803140,434510,416670
2,30,300,314770,4530,254960,34830,675950,502710,235380,258240,...,342570,34600,532890,812930,593030,211780,338130,616750,228200,303910
3,40,220440,414340,627670,286660,556780,786280,655200,454630,355090,...,743350,60,661180,379720,895890,282440,320,562010,413500,611500
4,50,362890,130,284080,220,634150,420,669490,10180,760560,...,820460,22670,526250,670230,574500,856160,601490,581300,655700,346300


#### Model 3

###### Item based
Recommend product based on the similarity between products.

In [78]:
# Read the user inventory table

print('Item Based Model')

dic_purchase = {}
with open(path_user_inventory,'rb') as f:
    lst_all = f.readlines()
    total_count = len(lst_all)
    current_count = 0
    for i in lst_all:
#         json_loads transfer file to dict
#         items() retune a list of dicts
        user_id, user_inventory = list(json.loads(i).items())[0]
#         if user_inventory is not empty, set an empty dict for user_id 
        if user_inventory != [] and user_inventory != None and user_inventory != {}:
            dic_purchase[user_id] = {}
            for playtime_info in user_inventory:
                appid = playtime_info.get('appid') # Get all appid in user_inventory
                if appid in set_valid_game_id:
                    dic_purchase[user_id].update({appid:1}) # Update user_id and all app each user played
        show_work_status(1,total_count,current_count)
        current_count+=1

Item Based Model
Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [87]:
# Set played game as 1 and unplayed game as 0
df_purchase = pd.DataFrame(dic_purchase).fillna(0)
purchase_matrix = df_purchase.values
lst_user_id = df_purchase.columns
lst_app_id = df_purchase.index

In [88]:
df_purchase.head()

,76561197960355015,76561197960385706,76561197960422789,76561197960464402,76561197960477514,76561197960562426,76561197960699459,76561197960893141,76561197961091730,76561197961243455,...,76561198258604825,76561198258639801,76561198258679897,76561198258749191,76561198259586242,76561198259723921,76561198259865770,76561198259925540,76561198259974871,76561198260675813
10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# Use linear kernel to calculate similarity of each row to all matrix 

dic_recomended_item_based = {}
total_count = purchase_matrix.shape[0]
current_count = 0

for index in range(total_count):
#     calculate similarity of each row to all matrix 
    cosine_similarities = linear_kernel(purchase_matrix[index:index+1], purchase_matrix).flatten()
#     take the negative number since we need the smallest as they are the most similar ones to target
    lst_related_app = (-cosine_similarities).argsort()[1:101]
#     Update dictionary with app id and the list of 100 most similar games
    dic_recomended_item_based.update({lst_app_id[index]:[lst_app_id[i] for i in lst_related_app]})
    show_work_status(1,total_count,current_count)
    current_count+=1

    

df_item_based_result = pd.DataFrame(dic_recomended_item_based).T
df_item_based_result.index.name = 'steam_appid'
df_item_based_result.reset_index(inplace=True)
df_item_based_result.to_sql('tbl_results_item_based',engine,if_exists='replace')

Status: [>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 100.00% 



In [545]:
lst_related_app

array([3166, 4123,  132,   22, 1399, 1477, 5387, 1567, 4476, 2817, 1867,
       3330, 1340, 3152, 3455, 1540, 2816, 2264, 5469, 4782, 1076, 3019,
       1471, 2420,  943, 3070, 5997, 2296, 1396, 1663, 1203, 3126,   20,
       1284, 4990, 1041,   24, 2399, 1903, 1702, 3151, 3370, 3110, 1658,
       1059, 1035, 2992, 1979, 1075, 1207, 5495, 2176, 1314, 3679, 3929,
       2274, 2437, 1094, 1407, 1411, 1424, 5743, 2001, 2547, 4813,  482,
       1800,  501,   14,   13,    9,   21,  296, 1458, 3390, 2436, 2888,
        215, 4971, 1468,   16, 1616, 3969, 1473, 3039, 3897,  398,  412,
        618, 1482, 4866, 4298, 3671,  554, 3577, 3572, 1754, 3880,  180,
       6115])

In [544]:
df_item_based_result.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,10,730,80,240,4000,550,304930,218620,340,620,...,238430,211820,200710,201790,236110,10090,40800,8980,4920,233720
1,20,70,50,130,730,340,550,320,220,4000,...,70000,220240,200510,9480,200260,273110,221910,17470,219150,35720
2,30,10,730,40,60,550,240,340,320,4000,...,10180,201790,291480,251570,207140,35720,211820,9480,236110,4920
3,40,40,30,60,730,550,240,340,320,80,...,40800,251570,200260,9480,291480,211820,35720,236110,207140,35140
4,50,130,70,20,730,550,340,320,220,4000,...,273110,200510,9480,70000,17470,220240,219150,291480,4920,35720


## Validation

Since we cannot have the users data in the coming several months and it takes time to test whether the model is correct, here we extract some users and check if they played similar games as a validation.

Now we have 3 tables of recommendation: 
* popularity_based_result
* content_based_result
* item_based_result

Based on users'favorite game, we can make recommendation.

In [92]:
df_popularity_based_results.head()

,steam_appid,owners
0,570,120750179
1,440,44290079
2,730,39311626
3,304930,35011290
4,578080,27699958


In [91]:
df_content_based_results.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,10,20,887520,837940,207230,683600,222880,670270,463920,743230,...,344340,19830,246820,652660,504460,763430,777850,803140,7940,843520
1,20,440,393410,207230,2720,209270,10,485310,511800,99300,...,342660,65800,222940,707010,288710,426190,473810,803140,434510,416670
2,30,300,314770,4530,254960,34830,675950,502710,235380,258240,...,342570,34600,532890,812930,593030,211780,338130,616750,228200,303910
3,40,220440,414340,627670,286660,556780,786280,655200,454630,355090,...,743350,60,661180,379720,895890,282440,320,562010,413500,611500
4,50,362890,130,284080,220,634150,420,669490,10180,760560,...,820460,22670,526250,670230,574500,856160,601490,581300,655700,346300


In [90]:
df_item_based_result.head()

,steam_appid,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,10,730,80,240,4000,550,352460,218620,340,620,...,4920,291550,209080,10180,251570,227300,70000,224600,223470,346900
1,20,70,50,130,730,340,550,320,220,4000,...,291480,4920,10090,251570,204300,228200,244850,201790,35140,227300
2,30,10,730,40,60,550,240,340,320,4000,...,35140,12120,10090,219150,220240,227300,200510,291550,17470,228200
3,40,40,30,60,730,550,240,340,320,80,...,4920,220240,12120,200510,10090,219150,17470,227300,228200,207610
4,50,130,70,20,730,550,340,320,220,4000,...,200260,10090,221910,35140,244850,228200,22370,251570,4560,201790


In [84]:
# Game information
df_valid_games.head()

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
0,10,Counter-Strike,game,9.99,2000-11-01,88.0,117473.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/10/...
1,20,Team Fortress Classic,game,4.99,1999-04-01,NaN,3653.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/20/...
2,30,Day of Defeat,game,4.99,2003-05-01,79.0,3576.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/30/...
3,40,Deathmatch Classic,game,4.99,2001-06-01,NaN,1439.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/40/...
4,50,Half-Life: Opposing Force,game,4.99,1999-11-01,NaN,4987.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/50/...


#### Test example
For example, if the user 76561197960422789's most played game is 730 (Counter-Strike: Global Offensive, a shooter game), we recommend

* Top 5 popular: Team Fortress 2, Dota 2, Counter-Strike: Global Offensive, Warframe, PLAYERUNKNOWN'S BATTLEGROUNDS
* Content_based: Counter-Strike: Condition Zero, Counter-Strike: Source, Counter-Strike Nexon: Zombies, DARIUSBURST Chronicle Saviours
* item_based: Left 4 Dead 2, Garry's Mod, PAYDAY 2, Warframe, Dead Realm

We can from the example that people who play CS may be willing to play:
* similar games like CS(other series) based on content-based recommendation 
* or other shooter games but with some cartoon or fantacy elements based on item-based recommendation

Noted that the item-based recommendation also recommend games from popularity tables, which indicates that people are willing to give a try on the most popular games. 

Combining the three models together can avoid problems such as cold start(no data to make recommendation) and pigeon holes(keep recommending the similar games) so that increasing users loyalty to platform. 

In [137]:
# Select the top 5 popular games
populartop5 = df_popularity_based_results['steam_appid'].iloc[0:6].values.flatten().tolist()
df_valid_games[df_valid_games['steam_appid'].isin(populartop5)]

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
23,440,Team Fortress 2,game,0.00,2007-10-10,92.0,521238.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/440...
26,570,Dota 2,game,0.00,2013-07-09,90.0,920096.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/570...
30,730,Counter-Strike: Global Offensive,game,0.00,2012-08-21,83.0,2677551.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/730...
4987,230410,Warframe,game,0.00,2013-03-25,71.0,198742.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/230...
24748,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,game,29.99,2017-12-21,86.0,865502.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/578...


In [138]:
df_user_favorite_app.head(10)

,steam_user_id,most_played_app
0,76561197960302536,NaN
1,76561197960323774,NaN
2,76561197960355015,10.0
3,76561197960385706,252950.0
4,76561197960422789,730.0
5,76561197960464402,730.0
6,76561197960477514,346110.0
7,76561197960562426,620.0
8,76561197960699459,252490.0
9,76561197960893141,730.0


In [96]:
df_valid_games[df_valid_games['steam_appid'] == 730]

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
30,730,Counter-Strike: Global Offensive,game,0.0,2012-08-21,83.0,2677551.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/730...


In [124]:
df_content_based_results[df_content_based_results['steam_appid'] == 730]

,steam_appid,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
25,730,377870,838900,240,80,273110,814480,525100,727790,500,...,479130,386740,3900,440280,220160,331650,773390,220,239820,882030


In [127]:
# Allocate the most similar games with app 730
simi730_content = df_content_based_results[df_content_based_results['steam_appid'] == 730].values.flatten().tolist()[1:6]
# We can see most the recommended game are from the same series or belongs to the same type.
df_valid_games[df_valid_games['steam_appid'].isin(simi730_content)]

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
7,80,Counter-Strike: Condition Zero,game,9.99,2004-03-01,65.0,12247.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/80/...
13,240,Counter-Strike: Source,game,9.99,2004-11-01,88.0,74394.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/240...
6532,273110,Counter-Strike Nexon: Zombies,game,0.00,2014-10-07,NaN,32983.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/273...
11956,377870,DARIUSBURST Chronicle Saviours,game,49.99,2015-12-03,80.0,415.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/377...
44107,838900,Cereal Soup,game,0.00,2018-06-02,NaN,316.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/838...


In [140]:
df_item_based_result[df_item_based_result['steam_appid'] == 730]

,steam_appid,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
22,730,4000,352460,550,218620,230410,240,105600,227940,252950,...,299360,48700,300,201790,223470,12120,200710,311210,274940,247730


In [141]:
# Allocate the most similar games with app 730
simi730_item = df_item_based_result[df_item_based_result['steam_appid'] == 730].values.flatten().tolist()[1:6]
# We can see most the recommended game are from the same type or the most popular games' table.
df_valid_games[df_valid_games['steam_appid'].isin(simi730_item)]

,steam_appid,name,type,initial_price,release_date,score,recommendation,windows,mac,linux,header_image
25,550,Left 4 Dead 2,game,9.99,2009-11-16,89.0,237612.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/550...
256,4000,Garry's Mod,game,9.99,2006-11-29,NaN,351217.0,1.0,1.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/400...
4488,218620,PAYDAY 2,game,9.99,2013-08-13,79.0,348546.0,1.0,0.0,1.0,https://steamcdn-a.akamaihd.net/steam/apps/218...
4987,230410,Warframe,game,0.00,2013-03-25,71.0,198742.0,1.0,0.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/230...
10490,352460,Dead Realm,game,14.99,2017-05-23,NaN,4631.0,1.0,1.0,0.0,https://steamcdn-a.akamaihd.net/steam/apps/352...
